In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!ls /datasets/aibubbsdata

Negative  Positive


In [3]:
!ls /datasets/bubbtest

Negative  Positive


In [4]:
image_height, image_width = 224, 224
batch_size = 512



train_ds = tf.keras.utils.image_dataset_from_directory(
    "/datasets/aibubbsdata",
    image_size = (image_height, image_width),
    batch_size = batch_size,
    subset = "training",
    seed = 49,
    validation_split = 0.2
  )

    
val_ds = tf.keras.utils.image_dataset_from_directory(
    "/datasets/aibubbsdata",
    image_size = (image_height, image_width),
    batch_size = batch_size,
    subset="validation",
    seed = 49,
    validation_split = 0.2
     )

test_ds = tf.keras.utils.image_dataset_from_directory(
   "/datasets/bubbtest",
    image_size = (image_height, image_width),
    batch_size = batch_size,
     )

Found 17886 files belonging to 2 classes.
Using 14309 files for training.
Found 17886 files belonging to 2 classes.
Using 3577 files for validation.
Found 1800 files belonging to 2 classes.


In [5]:
class_names = train_ds.class_names
print(class_names)



['Negative', 'Positive']


In [6]:
model = Sequential()


pretrained_model = tf.keras.applications.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    classes=2,
    classifier_activation='softmax'
)

for layer in pretrained_model.layers:
        layer.trainable=False
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))




ValueError: If using `weights` as `'imagenet'` with `include_top` as true, `classes` should be 1000Received: classes=2

In [ ]:
model.summary()

In [ ]:
checkpoint_path = "chk_EfficientNetV2S1_modi_model.h5"

try:
    model.load_weights(checkpoint_path)
    
except Exception as e:
    print(e)
    
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint(checkpoint_path, monitor='sparse_categorical_accuracy', save_best_only=True, save_weights_only=True,verbose=1, mode='auto')
early = EarlyStopping(monitor='sparse_categorical_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')    

In [ ]:
optimizer = Adam(learning_rate=0.005)
# model.compile(loss=
# tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=False,
#     ignore_class=None,
#     reduction=losses_utils.ReductionV2.AUTO,
#     name='sparse_categorical_crossentropy'
# ),
#     optimizer=optimizer,
#     metrics= ['MeanSquaredError','acc'])

model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


In [ ]:
history = model.fit(train_ds, validation_data= val_ds, epochs=10,callbacks=[checkpoint,early])

In [ ]:
import pandas as pd

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    

In [ ]:
 with pd.ExcelWriter('path_to_file.xlsx') as writer:
    hist_df.to_excel(writer)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os


#predictions = model.predict(test_ds)
#result = model.evaluate(test_ds)
#dict(zip(model.metrics_names, result))

predictions = model.predict(test_ds)
score = tf.nn.softmax(predictions)

for i in score:
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(i)], 100 * np.max(i))
    )


In [ ]:
result = model.evaluate(test_ds)
d = dict(zip(model.metrics_names, result))

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("xception.tflite", 'wb') as f:
  f.write(tflite_model)